# Brain Region Intensity Quantification

This notebook performs intensity quantification for brain regions using:
- registered_atlas.tiff (Allen atlas labels mask by brainreg, can be obtained using a reference image like autofluorescence at 561nm)
- downsampled.tiff (brain image that holds your signal)
- structures.csv (region names, from the allen atlas project)
- volumes.csv (region volumes, computed by brainreg)

In [1]:
import numpy as np
import pandas as pd
import tifffile
import dask.array as da
from dask.diagnostics import ProgressBar
from tqdm.notebook import tqdm

In [ ]:
# Read the image data
print("Loading atlas and brain image...")
atlas = tifffile.imread('registered_atlas.tiff')
brain = tifffile.imread('downsampled.tiff')

# Convert to dask arrays for parallel processing
atlas_da = da.from_array(atlas)
brain_da = da.from_array(brain)

# Read CSV files
print("Loading structure and volume data...")
structures_df = pd.read_csv('structures.csv')
volumes_df = pd.read_csv('volumes.csv')

# Create a mapping from structure name to ID
name_to_id = dict(zip(structures_df['name'], structures_df['id']))

### Calculation function for each region with Dask and atlas mask

In [3]:
def calculate_region_intensity(region_id):
    """Calculate total intensity for a given region ID using Dask."""
    # Create mask for the region
    mask = atlas_da == region_id
    
    # Calculate total intensity (multiply brain image by mask and sum)
    total_intensity = (brain_da * mask).sum()
    
    return total_intensity.compute()

### Actual calculation loop

In [ ]:
# Get unique region IDs from the atlas
region_ids = np.unique(atlas)
region_ids = region_ids[region_ids != 0]  # Remove background (0) from atlas

# Calculate intensities for all regions
print("Calculating intensities for each region... This will take some time.")
results = []

with tqdm(total=len(region_ids), desc="Processing regions", unit="region") as pbar:
    for region_id in region_ids:
        # Get region name from structures.csv
        region_name = structures_df[structures_df['id'] == region_id]['name'].iloc[0] \
            if len(structures_df[structures_df['id'] == region_id]) > 0 else f'Unknown_{region_id}'
        
        # Get volume from volumes.csv using the region name
        volume = volumes_df[volumes_df['structure_name'] == region_name]['total_volume_mm3'].iloc[0] \
            if len(volumes_df[volumes_df['structure_name'] == region_name]) > 0 else 0
        
        # Calculate total intensity
        total_intensity = calculate_region_intensity(region_id)
        
        # Calculate intensity per volume (semi-quantitative)
        intensity_per_volume = total_intensity / volume if volume > 0 else 0
        results.append({
            'structure_name': region_name,
            'total_intensity': total_intensity,
            'volume_mm3': volume,
            'intensity_per_volume': intensity_per_volume
        })
        pbar.update(1)
        pbar.set_postfix({"Last region": region_name, "Intensity": total_intensity})

### Dump into quanti.csv

In [ ]:
# Create final DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('quanti.csv', index=False)
print("Results saved to quanti.csv")
results_df.head()